In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from tqdm.notebook import tqdm

# from threading import Thread
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda')


In [3]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            # device_map='auto',
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            # temperature=0
            ).to(device)
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, infor,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # response, history = self.model.chat(self.tokenizer, prompt, history=[])
        # print('infor:', infor)
        # system_info =  infor.split('说“谢谢你的提问！”。')[-1].split('问题：')[0].strip()
        # human_info =  infor.split('问题：')[-1].strip()
        # time.sleep(1000)
        messages = [
                    # {'role':'system', 'content': system_info},
                    {'role':'user', 'content': infor}
                    ]
        # print('打印messages内容：', messages)
        # print('打印messages类型：', type(messages))
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors='pt').to(device)
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=1024
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
       
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [4]:
def init_db():
    persist_directory = r'vectordb/chroma/m3e-base'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/AI-ModelScope/m3e-base'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    # 加载缓存知识库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
    return vectordb

In [5]:
def init_model(vectordb):
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    template = """
        使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
        {context}
        问题：{question}
        """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=['context', 'question'], template=template)

    # 构造检索问答链
    qa_chain_ = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(
                    search_type="mmr", 
        search_kwargs={"k": 10, "fetch_k": 20}
        ),
        return_source_documents=True,
        chain_type_kwargs={'prompt': QA_CHAIN_PROMPT},
        # chain_type='stuff'
    )
    return qa_chain_

In [6]:
def init2_model(vectordb):
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    template = """
        使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
        上下文：{context}
        问题：{question}
        回答：
        """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=['context', 'question'], template=template)

    qa_chain_ = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT)

    return qa_chain_

In [7]:
# 加载数据
import pandas as pd
import csv
import os
# import re
# from tqdm import tqdm
# 初始化模型
vectordb = init_db()
model = init2_model(vectordb)

正从本地加载模型。。。。。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

模型加载完成！


In [22]:
datas = pd.read_csv('实体提取_改进.csv') 
datas_ = datas[datas['问题分类'] == '政策内容']
questions = datas_['问题']
labels = datas_['问题分类']
bodys = datas_['提取的实体']
# question = '意向受让方,登记受让意向,条件'
# 测试用实体来召回文本，并用召回的文本和问题回答
for question, body_ in tqdm(zip(questions, bodys), ncols=len(questions)):
    # 这里可能要将召回的问答链进行拆分，分两步
    question = '投标人或竞买人出现违约情形，保证金如何处理？'
    # body_ = '上海市规划和自然资源局关于印发《国有建设用地使用权招标拍卖挂牌出让投标竞买外汇保证金账户管理制度》的通知'
    body_ = '投标人,竞买人,违约,保证金,处理'
    call_data = vectordb.as_retriever(
        search_type="mmr", 
        search_kwargs={"k": 10, "fetch_k": 20}).get_relevant_documents(body_ )
    print([doc.metadata['title'] for doc in call_data])
    page_contents = ''
    for doc in call_data:
        page_contents += doc.page_content.strip() + '\n\n'
    
    response = model.run({'context': page_contents, 'question': question})
    print('response:', response)
    time.sleep(1000)
    # print('问题：', question)
    # print('回答：', response['result'])
    # print('来源文档：', response['source_documents'])

0it [00:00, ?it/s]

['【产权交易】上海联合产权交易所有限公司产权转让招投标实施办法（沪联产交〔2018〕49号）', '投标保证金提交与退还操作须知', '水运工程建设项目招标投标管理办法', '上海市规划和自然资源局关于国有建设用地使用权招标拍卖挂牌出让投标竞买保证金专户管理有关问题的通知', '政府采购货物和服务招标投标管理办法', '上海市建设工程招标投标管理办法', '【综合采购】上海市公共资源交易中心交易活动实施办法（试行）', '【司法拍卖】上海市公共资源拍卖中心拍卖当事人应当遵守的拍卖规则', '上海市发展和改革委员会关于印发《上海市公共资源交易目录(2020年版)》的通知', '工程建设项目施工招标投标办法']
infor: 
        使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
        上下文：第三十四条 中标人确定后，中标结果应当在联交所网站公告5日。同时，联交所应当将招标结果书面通知中标人和所有未中标人。第三十五条 中标通知书发出后，中标人的保证金可以转为产权交易价款。联交所应当在中标通知书发出之日起3个工作日内，将未中标人的保证金原额返还。第三十六条 招标人与中标人应当按照招标文件的要求和投标文件的承诺订立《产权交易合同》。招标文件和投标文件是《产权交易合同》的组成部分。第三十七条 招标文件发出后，招标人与投标人串通、随意终止招标程序、修改招标结果、不与确定的中标人签署产权交易合同的；招标人、投标人、招标代理机构之间相互串通，违反招标文件评标原则确定中标的；招标人与投标人以各种手段非法干预评标委员会正常评标，影响评标结果公正性的，应当承担相应的法律责任。第三十八条 招标代理机构在招标活动中泄露应当保密的与招标活动有关的情况和资料的；与招标人、投标人相互串通损害国家、社会公共利益或他人合法权益的，应当承担相应的法律责任。第三十九条 招标人、投标人、招标代理机构、受托机构故意提供不实数据、虚假证明资料或隐瞒重大事项，应当依法承担相应的法律责任。第四十条 招投标活动完毕，招标代理机构、评标委员会等相关机构，应当将招投标的所有文件送交联交所备案。第四十一条 本办法自发布之日起施行。上海联合产权交易所有限公司2018年10月22日

2，投标人必须按照招标文件的要求金额足额提交投标保

KeyboardInterrupt: 

In [16]:
第三十四条 中标人确定后，中标结果应当在联交所网站公告5日。同时，联交所应当将招标结果书面通知中标人和所有未中标人。第三十五条 中标通知书发出后，中标人的保证金可以转为产权交易价款。联交所应当在中标通知书发出之日起3个工作日内，将未中标人的保证金原额返还。第三十六条 招标人与中标人应当按照招标文件的要求和投标文件的承诺订立《产权交易合同》。招标文件和投标文件是《产权交易合同》的组成部分。第三十七条 招标文件发出后，招标人与投标人串通、随意终止招标程序、修改招标结果、不与确定的中标人签署产权交易合同的；招标人、投标人、招标代理机构之间相互串通，违反招标文件评标原则确定中标的；招标人与投标人以各种手段非法干预评标委员会正常评标，影响评标结果公正性的，应当承担相应的法律责任。第三十八条 招标代理机构在招标活动中泄露应当保密的与招标活动有关的情况和资料的；与招标人、投标人相互串通损害国家、社会公共利益或他人合法权益的，应当承担相应的法律责任。第三十九条 招标人、投标人、招标代理机构、受托机构故意提供不实数据、虚假证明资料或隐瞒重大事项，应当依法承担相应的法律责任。第四十条 招投标活动完毕，招标代理机构、评标委员会等相关机构，应当将招投标的所有文件送交联交所备案。第四十一条 本办法自发布之日起施行。上海联合产权交易所有限公司2018年10月22日

2，投标人必须按照招标文件的要求金额足额提交投标保证金，不得超额提交，更不得提供虚假、无效的票函。3、投标人不得以现金方式和使用经过背书转让的支票银行支票、银行本票、银行汇票提交投标保证金。4、银行保函的申请人必须是投标人，保证人必须是投标人基本账户的开户银行。5、银行保函采用示范样式（示范样式可向托管银行索取）。（二）退还方式以托管银行特种转账凭证返还原汇出银行账户。四，投标保证金利息计算和划付（一）计息利率．按退还投标保证金之日中国人民银行规定的活期存款利率计息。（二）划付方式：按投标保证金存放期间计算利息，退还投标保证金同时将利息划付至投标保证金的付款人账户。五、提交投标保证金程序（一）采用银行汇票、银行本票方式提交的：投标人携带公开招标网上“报名凭证”投标人基本账户开户许可证复印件，填写《投标保证金提交单》，由托管银行审核后开具《投标保证金提交确认函》。（二）采用支票提交的：投标人携带公开招标网上“报名凭证”、投标人基本账户开户许可证复印件，填写《投标保证金提交单》，由托管银行审核后在《投标保证金提交单》盖章。托管银行3个工作日后，确认支票到账，开具《投标保证金提交确认函》，投标人携带《投标保证金提交单》领取。（三）采用贷记凭证电汇网上支付方式提交的：投标保证金到账后，投标人携带公开招标网上“报名凭证”、投标人基本账户开户许可证复印件、入账回单复印件，填写《投标保证金提交单》，托管银行受理审核后开具《投标保证金提交确认函》。（四）采用银行保函的：投标人携带公开招标网上“报名凭证”银行保函、投标人基本账户开户许可证复印件，填写《投标保证奎提交单》，由托管银行审核后在《投标保证金提交单》盖章，银行不另行开具《投标保证金提交确认函》（五）邀请招标项目可用载有项目报建编号、标段号和投标保证金金额的投标邀请函（原件）替代“报名凭证”，邀请函必须加盖招标人、招标代理机构公章。

排名第一的中标候选人放弃中标、因不可抗力不能履行合同、不按照招标文件要求提交履约保证金，或者被查实存在影响中标结果的违法行为等情形，不符合中标条件的，招标人可以按照评标委员会提出的中标候选人名单排序依次确定其他中标候选人为中标人，也可以重新招标。第五十八条 中标人确定后，招标人应当及时向中标人发出中标通知书，并同时将中标结果通知所有未中标的投标人。第五十九条 招标人和中标人应当自中标通知书发出之日起30日内，按照招标文件和中标人的投标文件订立书面合同，合同的标的、价款、质量、履行期限等主要条款应当与招标文件和中标人的投标文件的内容一致。招标人和中标人不得再行订立背离合同实质性内容的其他协议。招标文件要求中标人提交履约保证金的，中标人应当按照招标文件的要求提交。履约保证金不得超过中标金额的10%。招标人最迟应当在书面合同签订后5日内向中标人和未中标的投标人退还投标保证金及银行同期存款利息。第六十条 中标候选人的经营、财务状况发生较大变化或者存在违法行为，招标人认为可能影响其履约能力的，应当在发出中标通知书前由原评标委员会按照招标文件规定的标准和方法审查确认。第六十一条 招标人应当自确定中标人之日起15日内，向具体负责本项目招标活动监督管理的交通运输主管部门提交招标投标情况的书面报告。招标投标情况书面报告主要内容包括：招标项目基本情况、投标人开标签到表、开标记录、监督人员名单、评标标准和方法、评标委员会评分表和汇总表、评标委员会推荐的中标候选人名单、中标人、经评标委员会签字的评标报告、评标结果公示、投诉处理情况等。第六十二条 中标人应当按照合同约定履行义务，完成中标项目。中标人不得向他人转让中标项目，也不得将中标项目肢解后分别向他人转让。中标人按照合同约定或者经招标人同意，可以将中标项目的部分非主体、非关键性工作分包给他人完成。

六、招标拍卖挂牌活动结束后，土地交易系统根据招拍挂结果自动生成《保证金退转名单》，土地交易市场于当日或下一个工作日将《保证金退转名单》、银行回单以及银行制作的入账信息表送交市规划资源管理部门。七、市规划资源管理部门应在招标拍卖挂牌活动结束后5个工作日内，对《保证金退转名单》、银行回单和入账信息表核对后，办理未中标人、未竞得人或未获得投标、竞买资格申请人的投标、竞买保证金的退款事宜。八、以人民币支付投标、竞买保证金之中标人、竞得人凭已支付出让金定金的国有建设用地使用权出让价款缴纳凭证，即《非税收入一般缴款书（收据）5》（以下简称“缴款凭证第五联”）及填写完整的《竞得人/中标人保证金退还申请表》向土地交易市场申请办理保证金退还手续。土地交易市场通过非税收入信息系统查询出让价款核销后，在“缴款凭证第五联”复印件和“退还申请表”上盖章确认，并于2个工作日内送交市规划资源管理部门。市规划资源管理部门核对后，在2个工作日内办理退款事宜。九、投标人、竞买人支付的人民币投标、竞买保证金按照相关规定通过原途径予以退还，但存在违约行为，按出让文件的规定保证金不予退还的情形除外。十、以外汇支付投标、竞买保证金之中标人、竞得人，未中标人、未竞得人或未获得投标、竞买资格的申请人，均按本市相关规定执行。十一、市规划资源管理部门按土地出让公告设立保证金明细账，并建立与银行定期核对的长效机制。本通知自2020年5月1日起施行，有效期至2025年4月30日，原《关于国有建设用地使用权招标拍卖挂牌出让投标竞买保证金专户管理有关问题的通知》（沪规土资财规〔2015〕305号）自本通知施行之时废止。上海市规划和自然资源局,2020年4月7日

第二十一条 招标人应当在招标文件中载明投标有效期。投标有效期从提交投标文件的截止之日起算。第二十二条 招标人在招标文件中要求投标人提交投标保证金的，投标保证金不得超过招标项目估算价的2%，最高不得超过80万元。投标保证金有效期应当与投标有效期一致。境内投标单位以现金或者支票形式提交的投标保证金应当从其基本账户转出。招标人不得挪用投标保证金。投标保证金鼓励采用银行保函形式收取。第二十三条 招标人设有最高投标限价的，应当在招标文件中明确最高投标限价或者最高投标限价的计算方法。招标人不得规定最低投标限价。第二十四条 招标人可以对已发出的资格预审文件或者招标文件进行必要的澄清或者修改，但有关澄清或者修改应当报原备案单位备案。澄清或者修改的内容可能影响资格预审申请文件或者投标文件编制的，招标人应当在提交资格预审申请文件截止时间至少3日前，或者投标截止时间至少15日前，以书面形式通知所有获取资格预审文件或者招标文件的潜在投标人；不足3日或者15日的，招标人应当顺延提交资格预审申请文件或者投标文件的截止时间。第二十五条  招标失败的，招标人应当在分析原因并采取相应措施后，重新进行招标。对招标文件中的招标范围、资格要求、评标办法等重点内容进行修改的，应当重新报民航地区管理局备案审核。重新招标后投标人仍少于三个的，属于必须审批、核准的工程建设项目，报经原审批、核准部门审批、核准后可以不再进行招标。第二十六条 招标人终止招标的，应当及时发布公告，或者以书面形式通知被邀请的或者已经获取资格预审文件、招标文件的潜在投标人。已经发售资格预审文件、招标文件或者已经收取投标保证金的，招标人应当及时退还所收取的资格预审文件、招标文件的费用，以及所收取的投标保证金及银行同期存款利息。

第五十六条  中标候选人的经营、财务状况发生较大变化或者存在违法行为，招标人认为可能影响其履约能力的，应当在发出中标通知书前由原评标委员会按照招标文件规定的标准和方法审查确认。第五十七条  招标人和中标人应当依照招标投标法和本条例的规定签订书面合同，合同的标的、价款、质量、履行期限等主要条款应当与招标文件和中标人的投标文件的内容一致。招标人和中标人不得再行订立背离合同实质性内容的其他协议。招标人最迟应当在书面合同签订后5日内向中标人和未中标的投标人退还投标保证金及银行同期存款利息。第五十八条  招标文件要求中标人提交履约保证金的，中标人应当按照招标文件的要求提交。履约保证金不得超过中标合同金额的10%。第五十九条  中标人应当按照合同约定履行义务，完成中标项目。中标人不得向他人转让中标项目，也不得将中标项目肢解后分别向他人转让。中标人按照合同约定或者经招标人同意，可以将中标项目的部分非主体、非关键性工作分包给他人完成。接受分包的人应当具备相应的资格条件，并不得再次分包。中标人应当就分包项目向招标人负责，接受分包的人就分包项目承担连带责任。第五章投诉与处理第六十条 投标人或者其他利害关系人认为招标投标活动不符合法律、行政法规规定的，可以自知道或者应当知道之日起10日内向有关行政监督部门投诉。投诉应当有明确的请求和必要的证明材料。就本条例第二十二条 、第四十四条 、第五十四条规定事项投诉的，应当先向招标人提出异议，异议答复期间不计算在前款规定的期限内。第六十一条  投诉人就同一事项向两个以上有权受理的行政监督部门投诉的，由最先收到投诉的行政监督部门负责处理。行政监督部门应当自收到投诉之日起3个工作日内决定是否受理投诉，并自受理投诉之日起30个工作日内作出书面处理决定；需要检验、检测、鉴定、专家评审的，所需时间不计算在内。

第三十八条  投标人在投标截止时间前撤回已提交的投标文件的，采购人或者采购代理机构应当自收到投标人书面撤回通知之日起5个工作日内，退还已收取的投标保证金，但因投标人自身原因导致无法及时退还的除外。采购人或者采购代理机构应当自中标通知书发出之日起5个工作日内退还未中标人的投标保证金，自采购合同签订之日起5个工作日内退还中标人的投标保证金或者转为中标人的履约保证金。采购人或者采购代理机构逾期退还投标保证金的，除应当退还投标保证金本金外，还应当按中国人民银行同期贷款基准利率上浮20％后的利率支付超期资金占用费，但因投标人自身原因导致无法及时退还的除外。第四章开标、评标第三十九条 开标应当在招标文件确定的提交投标文件截止时间的同一时间进行。开标地点应当为招标文件中预先确定的地点。采购人或者采购代理机构应当对开标、评标现场活动进行全程录音录像。录音录像应当清晰可辨，音像资料作为采购文件一并存档。第四十条 开标由采购人或者采购代理机构主持，邀请投标人参加。评标委员会成员不得参加开标活动。第四十一条 开标时，应当由投标人或者其推选的代表检查投标文件的密封情况；经确认无误后，由采购人或者采购代理机构工作人员当众拆封，宣布投标人名称、投标价格和招标文件规定的需要宣布的其他内容。投标人不足3家的，不得开标。第四十二条 开标过程应当由采购人或者采购代理机构负责记录，由参加开标的各投标人代表和相关工作人员签字确认后随采购文件一并存档。投标人代表对开标过程和开标记录有疑义，以及认为采购人、采购代理机构相关工作人员有需要回避的情形的，应当场提出询问或者回避申请。采购人、采购代理机构对投标人代表提出的询问或者回避申请应当及时处理。投标人未参加开标的，视同认可开标结果。

第二十六条招标人可以在招标文件中要求投标人提交投标担保。投标担保可以采用投标保函或者投标保证金的方式。投标保证金可以使用支票、银行汇票等，一般不得超过投标总价的2％，最高不得超过50万元。投标人应当按照招标文件要求的方式和金额，将投标保函或者投标保证金随投标文件提交招标人。第二十七条投标人应当在招标文件要求提交投标文件的截止时间前，将投标文件密封送达投标地点。招标人收到投标文件后，应当向投标人出具标明签收人和签收时间的凭证，并妥善保存投标文件。在开标前，任何单位和个人均不得开启投标文件。在招标文件要求提交投标文件的截止时间后送达的投标文件，为无效的投标文件，招标人应当拒收。提交投标文件的投标人少于3个的，招标人应当依法重新招标。第二十八条投标人在招标文件要求提交投标文件的截止时间前，可以补充、修改或者撤回已提交的投标文件。补充、修改的内容为投标文件的组成部分，并应当按照本办法第二十七条第一款的规定送达、签收和保管。在招标文件要求提交投标文件的截止时间后送达的补充或者修改的内容无效。第二十九条两个以上施工企业可以组成一个联合体，签订共同投标协议，以一个投标人的身份共同投标。联合体各方均应当具备承担招标工程的相应资质条件。相同专业的施工企业组成的联合体，按照资质等级低的施工企业的业务许可范围承揽工程。招标人不得强制投标人组成联合体共同投标，不得限制投标人之间的竞争。第三十条投标人不得相互串通投标，不得排挤其他投标人的公平竞争，损害招标人或者其他投标人的合法权益。投标人不得与招标人串通投标，损害国家利益、社会公共利益或者他人的合法权益。禁止投标人以向招标人或者评标委员会成员行贿的手段谋取中标。第三十一条投标人不得以低于其企业成本的报价竞标，不得以他人名义投标或者以其他方式弄虚作假，骗取中标。第四章 开标、评标和中标。

中标人、竞得人支付的投标、竞买保证金抵作土地出让价款；其他投标人、竞买人支付的投标、竞买保证金，出让人必须在招标拍卖挂牌活动结束后5个工作日内予以退还，不计利息。第二十二条 招标拍卖挂牌活动结束后，出让人应在10个工作日内将招标拍卖挂牌出让结果在土地有形市场或者指定的场所、媒介公布。出让人公布出让结果，不得向受让人收取费用。第二十三条 受让人依照国有建设用地使用权出让合同的约定付清全部土地出让价款后，方可申请办理土地登记，领取国有建设用地使用权证书。未按出让合同约定缴清全部土地出让价款的，不得发放国有建设用地使用权证书，也不得按出让价款缴纳比例分割发放国有建设用地使用权证书。第二十四条 应当以招标拍卖挂牌方式出让国有建设用地使用权而擅自采用协议方式出让的，对直接负责的主管人员和其他直接责任人员依法给予处分；构成犯罪的，依法追究刑事责任。第二十五条 中标人、竞得人有下列行为之一的，中标、竞得结果无效；造成损失的，应当依法承担赔偿责任：（一）提供虚假文件隐瞒事实的；（二）采取行贿、恶意串通等非法手段中标或者竞得的。第二十六条 国土资源行政主管部门的工作人员在招标拍卖挂牌出让活动中玩忽职守、滥用职权、徇私舞弊的，依法给予处分；构成犯罪的，依法追究刑事责任。第二十七条 以招标拍卖挂牌方式租赁国有建设用地使用权的，参照本规定执行。第二十八条 本规定自2007年11月1日起施行。

违反前两款规定的，相关投标均无效。第三十五条  投标人撤回已提交的投标文件，应当在投标截止时间前书面通知招标人。招标人已收取投标保证金的，应当自收到投标人书面撤回通知之日起5日内退还。投标截止后投标人撤销投标文件的，招标人可以不退还投标保证金。第三十六条  未通过资格预审的申请人提交的投标文件，以及逾期送达或者不按照招标文件要求密封的投标文件，招标人应当拒收。招标人应当如实记载投标文件的送达时间和密封情况，并存档备查。第三十七条  招标人应当在资格预审公告、招标公告或者投标邀请书中载明是否接受联合体投标。招标人接受联合体投标并进行资格预审的，联合体应当在提交资格预审申请文件前组成。资格预审后联合体增减、更换成员的，其投标无效。联合体各方在同一招标项目中以自己名义单独投标或者参加其他联合体投标的，相关投标均无效。第三十八条  投标人发生合并、分立、破产等重大变化的，应当及时书面告知招标人。投标人不再具备资格预审文件、招标文件规定的资格条件或者其投标影响招标公正性的，其投标无效。第三十九条  禁止投标人相互串通投标。有下列情形之一的，属于投标人相互串通投标：(一)投标人之间协商投标报价等投标文件的实质性内容；(二)投标人之间约定中标人；(三)投标人之间约定部分投标人放弃投标或者中标；(四)属于同一集团、协会、商会等组织成员的投标人按照该组织要求协同投标；(五)投标人之间为谋取中标或者排斥特定投标人而采取的其他联合行动。第四十条  有下列情形之一的，视为投标人相互串通投标：(一)不同投标人的投标文件由同一单位或者个人编制；(二)不同投标人委托同一单位或者个人办理投标事宜；(三)不同投标人的投标文件载明的项目管理成员为同一人；(四)不同投标人的投标文件异常一致或者投标报价呈规律性差异；(五)不同投标人的投标文件相互混装；(六)不同投标人的投标保证金从同一单位或者个人的账户转出。

投标有效期内投标人撤销投标文件的，采购人或者采购代理机构可以不退还投标保证金。第二十四条 招标文件售价应当按照弥补制作、邮寄成本的原则确定，不得以营利为目的，不得以招标采购金额作为确定招标文件售价的依据。第二十五条 招标文件、资格预审文件的内容不得违反法律、行政法规、强制性标准、政府采购政策，或者违反公开透明、公平竞争、公正和诚实信用原则。有前款规定情形，影响潜在投标人投标或者资格预审结果的，采购人或者采购代理机构应当修改招标文件或者资格预审文件后重新招标。第二十六条 采购人或者采购代理机构可以在招标文件提供期限截止后，组织已获取招标文件的潜在投标人现场考察或者召开开标前答疑会。组织现场考察或者召开答疑会的，应当在招标文件中载明，或者在招标文件提供期限截止后以书面形式通知所有获取招标文件的潜在投标人。第二十七条 采购人或者采购代理机构可以对已发出的招标文件、资格预审文件、投标邀请书进行必要的澄清或者修改，但不得改变采购标的和资格条件。澄清或者修改应当在原公告发布媒体上发布澄清公告。澄清或者修改的内容为招标文件、资格预审文件、投标邀请书的组成部分。澄清或者修改的内容可能影响投标文件编制的，采购人或者采购代理机构应当在投标截止时间至少15日前，以书面形式通知所有获取招标文件的潜在投标人；不足15日的，采购人或者采购代理机构应当顺延提交投标文件的截止时间。澄清或者修改的内容可能影响资格预审申请文件编制的，采购人或者采购代理机构应当在提交资格预审申请文件截止时间至少3日前，以书面形式通知所有获取资格预审文件的潜在投标人；不足3日的，采购人或者采购代理机构应当顺延提交资格预审申请文件的截止时间。第二十八条 投标截止时间前，采购人、采购代理机构和有关人员不得向他人透露已获取招标文件的潜在投标人的名称、数量以及可能影响公平竞争的有关招标投标的其他情况。

进场交易的建设工程招标开始前，招标人应当向市招标投标办或者区建设行政管理部门提交风险承诺书。政府投资的建设工程施工招标应当具备施工图设计文件，施工监理招标应当具备经批准的初步设计文件。第九条 （工程总承包再发包）工程总承包单位依法将其承接的勘察、设计或者施工依法再发包给具有相应资质企业的，可以采用招标发包或者直接发包；相应的设计、施工总承包企业可以依法将部分专业工程分包。第十条 （批量招标和预选招标）招标人在同一时间段实施多个同类工程的，可以采用批量招标的方式进行招标。应急抢险工程以及经常发生的房屋修缮、园林绿化养护、市政设施和水利设施维修等工程，可以采用预选招标的方式进行招标。第十一条 （标段划分）招标人应当根据建设工程特点合理划分标段，不得利用标段划分降低投标人资格条件。建设工程招标标段的划分标准，由市建设行政管理部门会同有关行政管理部门确定。第十二条 （不招标情形）通过招标、竞争性谈判、竞争性磋商等竞争方式取得建设项目的建设单位，具有勘察、设计、施工资质并由其自行进行该项目勘察、设计、施工的，该项目的勘察、设计、施工可免于招标。依法必须进行招标的建设工程，有下列情形之一且原中标人仍具备承包能力的，可以不进行招标：（一）已建成工程进行改、扩建或者技术改造，需由原中标人进行勘察、设计，否则将影响项目功能配套性的；（二）在建工程追加的附属小型工程或者主体加层工程，需由原中标人进行勘察、设计、施工、监理的，追加的全部附属小型工程在原项目审批范围内，造价累计不超过原中标价的30%且低于1000万元的；（三）与在建工程结构紧密相连，受施工场地限制且安全风险大，须由原中标人进行勘察、设计、施工、监理，并经本市专项技术评审专家库中抽取的专家论证的；（四）法律、法规、规章规定的其他情形。招标人为适用前款规定弄虚作假而规避招标的，按照《中华人民共和国招标投标法》规避招标的有关规定处理。

投标人根据招标文件载明的项目实际情况，拟在中标后将中标项目的部分非主体、非关键性工作进行分包的，应当在投标文件中载明。第三十七条招标人可以在招标文件中要求投标人提交投标保证金。投标保证金除现金外，可以是银行出具的银行保函、保兑支票、银行汇票或现金支票。投标保证金不得超过项目估算价的百分之二，但最高不得超过八十万元人民币。投标保证金有效期应当与投标有效期一致。投标人应当按照招标文件要求的方式和金额，将投标保证金随投标文件提交给招标人或其委托的招标代理机构。依法必须进行施工招标的项目的境内投标单位，以现金或者支票形式提交的投标保证金应当从其基本账户转出。第三十八条投标人应当在招标文件要求提交投标文件的截止时间前，将投标文件密封送达投标地点。招标人收到投标文件后，应当向投标人出具标明签收人和签收时间的凭证，在开标前任何单位和个人不得开启投标文件。在招标文件要求提交投标文件的截止时间后送达的投标文件，招标人应当拒收。依法必须进行施工招标的项目提交投标文件的投标人少于三个的，招标人在分析招标失败的原因并采取相应措施后，应当依法重新招标。重新招标后投标人仍少于三个的，属于必须审批、核准的工程建设项目，报经原审批、核准部门审批、核准后可以不再进行招标；其他工程建设项目，招标人可自行决定不再进行招标。第三十九条投标人在招标文件要求提交投标文件的截止时间前，可以补充、修改、替代或者撤回已提交的投标文件，并书面通知招标人。补充、修改的内容为投标文件的组成部分。第四十条在提交投标文件截止时间后到招标文件规定的投标有效期终止之前，投标人不得撤销其投标文件，否则招标人可以不退还其投标保证金。第四十一条在开标前，招标人应妥善保管好已接收的投标文件、修改或撤回通知、备选投标方案等投标资料。第四十二条两个以上法人或者其他组织可以组成一个联合体，以一个投标人的身份共同投标。

中标人不与招标人订立合同的，投标保证金不予退还并取消其中标资格，给招标人造成的损失超过投标保证金数额的，应当对超过部分予以赔偿；没有提交投标保证金的，应当对招标人的损失承担赔偿责任。招标人无正当理由不与中标人签订合同，给中标人造成损失的，招标人应当给予赔偿。第四十七条招标文件要求中标人提交履约担保的，中标人应当提交。招标人应当同时向中标人提供工程款支付担保。第五章，罚则第四十八条有违反《招标投标法》行为的，县级以上地方人民政府建设行政主管部门应当按照《招标投标法》的规定予以处罚。第四十九条招标投标活动中有《招标投标法》规定中标无效情形的，由县级以上地方人民政府建设行政主管部门宣布中标无效，责令重新组织招标，并依法追究有关责任人责任。第五十条应当招标未招标的，应当公开招标未公开招标的，县级以上地方人民政府建设行政主管部门应当责令改正，拒不改正的，不得颁发施工许可证。第五十一条招标人不具备自行办理施工招标事宜条件而自行招标的，县级以上地方人民政府建设行政主管部门应当责令改正，处1万元以下的罚款。第五十二条评标委员会的组成不符合法律、法规规定的，县级以上地方人民政府建设行政主管部门应当责令招标人重新组织评标委员会。第五十三条招标人未向建设行政主管部门提交施工招标投标情况书面报告的，县级以上地方人民政府建设行政主管部门应当责令改正。第六章，附则第五十四条工程施工专业分包、劳务分包采用招标方式的，参照本办法执行。第五十五条招标文件或者投标文件使用两种以上语言文字的，必须有一种是中文；如对不同文本的解释发生异议的，以中文文本为准。用文字表示的金额与数字表示的金额不一致的，以文字表示的金额为准。第五十六条涉及国家安全、国家秘密、抢险救灾或者属于利用扶贫资金实行以工代赈、需要使用农民工等特殊情况，不适宜进行施工招标的工程，按照国家有关规定可以不进行施工招标。

第二十条【投标澄清与说明】响应方应当根据评标委员会（评审小组）的要求，在不超出投标（响应）文件的范围或改变投标（响应）文件实质性内容的基础上，对投标（响应）文件中含义不明确的内容进行必要的澄清或说明。第二十一条【结果通知】评标（评审）结束后，发起方、中介机构可以通过市交易平台编辑中标候选人信息和成交结果信息并完成签章后，发布结果通知。发起方选择结果公示的，市交易中心对公示信息完成核验后，及时通过市交易中心门户网站等媒介发布。第二十二条【合同签署】发起方和中标人（成交单位）应当在规定的时间内按照交易公告、交易文件和中标人（成交单位）的投标（响应）文件订立书面交易合同，并将交易合同电子件通过市交易平台上传。第二十三条【特殊项目】涉密、远程异地等特殊项目需要进入市交易中心开展交易的，根据一事一议原则，由发起方或项目所在地公共资源交易中心向市交易中心提交书面申请。市交易中心在收到书面申请后三个工作日内作出同意或不同意的决定。市交易中心作出同意决定的，组织发起方根据项目特性，进场开展线上或线下交易。第三章 服务与规范第二十四条【服务保障】市交易中心负责开评标现场和相关设施设备的保障及维护。第二十五条【交易见证】市交易中心为发起方和相关主管部门提供交易活动的见证服务。需要现场见证、出具见证报告或音视频刻录服务的，发起方应当按照见证服务相关细则向市交易中心提交书面申请。第二十六条【行为规范】市交易中心负责维护交易秩序，规范交易主体、评标（评审）专家以及监督人在场内开展交易活动时的行为，对违反承诺及市交易中心制度规定的，市交易中心按相关规定处理。市交易中心制定信用记分评估标准，对发起方、响应方、中介机构及其从业人员和评标（评审）专家在市交易中心开展交易活动实施信用记分评估工作。第二十七条【法律责任】发起方应当对上传或填报信息材料的真实性、完整性、准确性作出承诺并承担责任。

竞买人（网上竞买人）未竞得拍卖标的的，拍卖人应当退还竞买人交付的保证金。十二、拍卖成交后，买受人交付的保证金即转为定金。买受人（网上买受人）须在约定期限内付清成交价款和佣金，拍卖人须按照约定交付标的物。司法委托拍卖标的的交付按司法程序办理。买受人未能按约定受领拍卖标的的，自约定交付之时起即对标的承担风险责任，并承担由此产生的费用。拍卖人、买受人未按约定履行义务的，应依法承担违约责任。十三、拍卖标的须依法办理证照变更、权属转受让登记的，买受人应持拍卖人出具的成交证明和有关资料，会同委托人向有关管理机关办理手续；拍卖人须提供必要的协助服务。买受人应当按照法律、法规的规定处置其购得的拍卖标的，依法履行相关义务。十四、拍卖活动当事人应当遵守拍卖法律、法规的规定和拍卖现场秩序，不得喧哗吵闹，不得阻挠竞买人竞价，不得阻碍拍卖师进行正常工作。当事人之间不得恶意串通，损害他人利益。更不能有操纵、垄断等违法行为。一经发现将依法追究其法律责任。由于互联网可能出现不稳定情况，不排除网络拍卖发生故障（包括但不限于网络故障、电路故障、系统故障）以及被网络黑客恶意攻击，网络竞买人必须充分估计上述原因导致网上竞价不同于现场竞价所带来的风险，如果发生上述情况以及本人操作差错所造成的损失由网络竞买人承担。网络竞买人一旦参与网上竞价，即认同和接受网络拍卖的客观状态、拍卖会现场传导给客户端的拍卖标的图文信息和音频视频直播信号、拍卖师的主持、拍卖成交价的确认。十五、委托人、竞买人或其他人员需对拍卖活动情况进行摄影、摄像和录音的，须事先征得拍卖人同意并办理登记备案。非经拍卖人同意，任何人不得擅自发布拍卖活动信息。十六、买受人未按照约定付款的，应当承担违约责任，或者由拍卖人征得委托人的同意，将拍卖标的再行拍卖。拍卖标的再行拍卖的，原买受人应当支付第一次拍卖中本人及委托人应当支付的佣金。

第四十四条 （招标人的决策约束）本市国有资产监督管理部门和有关行政管理部门应当要求国有企业事业单位建立建设工程招标投标活动的决策约束制度；资格预审、投标筛选、定标等事项应当纳入决策约束制度。国有企业事业单位负责人的考核中，应当包括有关建设工程招标投标情况以及决策约束制度落实情况。第四十五条 （投标保证金）招标人在招标文件中要求投标人提交投标保证金的，投标人可以采取银行保函、保证保险等方式提供保证。第五章法律责任第四十六条（违反进场交易、划分标段、资格预审、组建评标委员会、改变招标投标文件实质性内容规定的处罚）有下列情形之一的，由建设行政管理部门责令改正，处3万元以上10万元以下的罚款：（一）违反本办法第四条 规定，政府投资的建设工程，以及国有企业事业单位使用自有资金且国有资产投资者实际拥有控制权的建设工程，达到法定招标规模标准，招标人未进入招标投标交易场所进行全过程招标投标活动的；（二）违反本办法第十一条 规定，招标人未按照规定划分标段的；（三）违反本办法第三十六条第二款规定，招标人对中标候选人提出改变招标文件和投标文件实质性内容要求的。第四十七条（投标文件时限违反规定的法律责任）违反本办法第十九条第一款规定，投标文件编制期限最短少于20日的，由建设行政管理部门按照《中华人民共和国招标投标法实施条例》第六十四条的规定处罚。违反本办法第十九条第二款规定，工程总承包招标中投标文件编制期限最短少于30日的，由建设行政管理部门责令改正，处2万元以上10万元以下的罚款。第四十八条（招标人以不合理条件限制、排斥潜在投标人的法律责任）违反本办法第十四条 第一款第十五条 第二款第二十三条规定，招标人以不合理条件限制、排斥潜在投标人的，由建设行政管理部门按照《中华人民共和国招标投标法》第五十一条的规定处罚。

第三十一条  招标人终止招标的，应当及时发布公告，或者以书面形式通知被邀请的或者已经获取资格预审文件、招标文件的潜在投标人。已经发售资格预审文件、招标文件或者已经收取投标保证金的，招标人应当及时退还所收取的资格预审文件、招标文件的费用，以及所收取的投标保证金及银行同期存款利息。第三十二条   招标人不得以不合理的条件限制、排斥潜在投标人或者投标人。招标人有下列行为之一的，属于以不合理条件限制、排斥潜在投标人或者投标人：(一)就同一招标项目向潜在投标人或者投标人提供有差别的项目信息；(二)设定的资格、技术、商务条件与招标项目的具体特点和实际需要不相适应或者与合同履行无关；(三)依法必须进行招标的项目以特定行政区域或者特定行业的业绩、奖项作为加分条件或者中标条件；(四)对潜在投标人或者投标人采取不同的资格审查或者评标标准；(五)限定或者指定特定的专利、商标、品牌、原产地或者供应商；(六)依法必须进行招标的项目非法限定潜在投标人或者投标人的所有制形式或者组织形式；(七)以其他不合理条件限制、排斥潜在投标人或者投标人。第三章投标第三十三条  投标人参加依法必须进行招标的项目的投标，不受地区或者部门的限制，任何单位和个人不得非法干涉。第三十四条  与招标人存在利害关系可能影响招标公正性的法人、其他组织或者个人，不得参加投标。单位负责人为同一人或者存在控股、管理关系的不同单位，不得参加同一标段投标或者未划分标段的同一招标项目投标。违反前两款规定的，相关投标均无效。第三十五条  投标人撤回已提交的投标文件，应当在投标截止时间前书面通知招标人。招标人已收取投标保证金的，应当自收到投标人书面撤回通知之日起5日内退还。投标截止后投标人撤销投标文件的，招标人可以不退还投标保证金。第三十六条  未通过资格预审的申请人提交的投标文件，以及逾期送达或者不按照招标文件要求密封的投标文件，招标人应当拒收。

序号交易类别交易内容一工程建设项目招标投标建筑工程房屋建筑工程、市政公用工程、机电工程、园林（廊道）、绿化、人防等其它工程的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料的采购，工程建设、运维保养有关的招标代理、代建、监测检验、维修保养等交通工程铁路工程、公路工程、水运工程、城市道路（含高架、地面道路）、轨道交通、枢纽场站、公共停车场库、公交线路站点等公交系统工程、民用机场工程、桥梁工程、隧道工程、其它交通运输或配套工程的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料的采购，工程建设、运维保养有关的招标代理、代建、监测检验、维修保养等水务工程水利工程、供水工程、排水工程、泵站工程、水文工程、其它水务工程的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料的采购，工程建设、运维保养有关的招标代理、代建、监测检验、维修保养等电力与能源工程火电工程、输配电工程、供电用电工程、新能源工程（风电、生物质能）、石油天然气管网工程、其它电力与能源工程、与工程建设有关的重要设备、材料的采购，工程建设、运维保养有关的招标代理、代建、监测检验、维修保养等公用事业基础设施科技、教育、广播电视等工程、体育、文化、旅游等工程、卫生、林业、环卫、社会福利等工程、政府保障性住房等工程、其他公用事业工程的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料的采购，工程建设、运维保养有关的招标代理、代建、监测检验、维修保养等序号交易类别交易内容二土地使用权出让商业、旅游、娱乐、商品住宅等经营性用地出让、工业、科研设计用地及其他用地三国有产权交易产（股）权交易履行出资人职责的机构、国有及国有控股企业、国有实际控制企业转让其对企业各种形式出资所形成权益的行为，国有及国有控股企业、国有实际控制企业增加资本的行为，行政事业单位分立、撤销、合并、改制、隶属关系改变等原因发生的产权和股权转让，金融企业国有出资所形成权益转让以及增加资本的行为知识产权交易企业、事业单位国有知识产权项目（含专利权转让，商标所有权、使用权转让，其他知识产权转让等），政府相关部门组织的专利技术转让、其他知识产权转让等国有技术产权交易项目物权交易国有及国有控股企业、国有实际控制企业的重大资产转让行为（含公房、公车等闲置资产出售、租赁，有一定价值的废旧、淘汰物品处置，其它物品处置），行政机关、团体、事业单位资产处置等产权、物权交易项目（含公房、公车等闲置资产出售、租赁，有一定价值的废旧、淘汰物品处置，各类上交物品处置，其他物品处置）四政府采购国家机关、事业单位和团体组织使用财政性资金采购依法制定的集中采购目录以内的或者采购限额标准以上的工程、货物及服务五机电设备国内招标企业使用国有资金采购的机电设备，包括但不限于机械加工、自动化生产、工业机器人等产业类设备，计算机、打印机、复印机、传真机、通讯等信息类设备，家用电器、家用加工机械、汽车电子化产品、健身运动机械等民生类设备六农村集体产权交易农村集体土地经营权流转、农村集体经营性资产出租、农村集体资产股权转让序号交易类别交易内容七无形资产交易基础设施和公用事业特许经营权授予，市政公用设施及公共场地使用权、承包经营权、冠名权有偿转让，经营性国有资产租赁及经营性权益转让，政府举办的体育赛事承办运营权、冠名及赞助权、转播权等无形资产开发八技术交易企业、事业单位获得财政资金支持的科技成果交易、技术服务交易，科研事业单位职务科技成果交易、技术服务交易（含技术转让、技术许可、技术作价入股、技术服务采购、技术合作开发等），以及政府相关部门组织的技术成果转让、技术成果推广、技术成果供需对接等非关系到国家经济安全、国防安全和国家秘密的科技成果九碳排放交易政府签发的碳配额、碳信用、其他温室气体减排量及相关产品十用能权交易用能单位或其他能源消耗总量和强度“双控”目标责任主体之间年度综合能源消费量指标，以吨标准煤为单位十一司法机关和行政执法部门开展的涉诉、抵债或罚没资产处置人民法院和公安机关、海关等行政执法部门依法处置的资产（含车辆、船舶、飞行器、物资、机械设备、生产资源、生活用品等动产，房屋、在建工程等不动产，股权、各类权益、商标、专利等无形资产，以及法律法规允许变价处置、转让的其他财产）

第八十二条中标人将中标项目转让给他人的，将中标项目肢解后分别转让给他人的，违法将中标项目的部分主体、关键性工作分包给他人的，或者分包人再次分包的，转让、分包无效，有关行政监督部门处转让、分包项目金额千分之五以上千分之十以下的罚款；有违法所得的，并处没收违法所得；可以责令停业整顿；情节严重的，由工商行政管理机关吊销营业执照。第八十三条招标人与中标人不按照招标文件和中标人的投标文件订立合同的，合同的主要条款与招标文件、中标人的投标文件的内容不一致，或者招标人、中标人订立背离合同实质性内容的协议的，有关行政监督部门责令改正；可以处中标项目金额千分之五以上千分之十以下的罚款。第八十四条中标人不履行与招标人订立的合同的，履约保证金不予退还，给招标人造成的损失超过履约保证金数额的，还应当对超过部分予以赔偿；没有提交履约保证金的，应当对招标人的损失承担赔偿责任。中标人不按照与招标人订立的合同履行义务，情节严重的，有关行政监督部门取消其二至五年参加招标项目的投标资格并予以公告，直至由工商行政管理机关吊销营业执照。因不可抗力不能履行合同的，不适用前两款规定。第八十五条招标人不履行与中标人订立的合同的，应当返还中标人的履约保证金，并承担相应的赔偿责任；没有提交履约保证金的，应当对中标人的损失承担赔偿责任。因不可抗力不能履行合同的，不适用前款规定。第八十六条依法必须进行施工招标的项目违反法律规定，中标无效的，应当依照法律规定的中标条件从其余投标人中重新确定中标人或者依法重新进行招标。中标无效的，发出的中标通知书和签订的合同自始没有法律约束力，但不影响合同中独立存在的有关解决争议方法的条款的效力。

'投标保证金提交与退还操作须知'

In [ ]:
来源文档： [Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-12', 'title': '【产权交易】上海联合产权交易所有限公司企业国有产权转让登记受让意向操作细则（沪联产交〔2020〕45号)'}, page_content='第十三条 登记的意向受让方不符合信息披露公告中的受让方资格条件，或提出带有附加条件的受让要求，或提交的材料不符合齐全性和规范性要求的，联交所以书面形式告知意向受让方。意向受让方应当在收到通知的次日起2个工作日内按要求做出调整。第十四条 联交所在信息披露期满后5个工作日内，就意向受让方是否符合受让条件以《受让资格反馈函》反馈至转让方。第十五条 转让方应当在收到联交所《受让资格反馈函》次日起5个工作日内予以书面回复，如认为意向受让方不符合公布的受让方资格条件的，应当向联交所提出书面意见，说明理由并提交相关证明材料。转让方逾期未予书面回复的，视为同意联交所作出的资格确认意见。第十六条 转让方与联交所对意向受让方资格确认意见不一致的，应当就争议事项征询转让行为批准单位或国资监管机构意见，联交所根据转让行为批准单位或国资监管机构的书面意见确定意向受让方是否符合受让条件。第十七条 经征询转让方意见后联交所向意向受让方出具交易安排通知函件或《受让资格确认结果通知》，并同时抄送转让方。第十八条 意向受让方应当在交易安排通知函件规定的时限内，将交易保证金交纳至联交所指定账户。意向受让方按规定交纳交易保证金后获得资格确认。意向受让方未按规定交纳保证金的，视为放弃受让资格。第十九条 未获得资格确认的意向受让方如有异议，在收到《受让资格确认结果通知》次日起2个工作日内可通过联交所向转让方提出书面意见，说明提出异议的理由并提交相关证明材料。意向受让方的异议不影响相关产权交易项目的进行。第二十条 转让项目选择开放式报价方式的，按照联交所开放式报价相关规定执行。第二十一条 本细则自发布之日起施行。上海联合产权交易所有限公司2020年7月21日'), Document(metadata={'infosource': '信息来源：上海市', 'time': '发布时间：2020-12-22', 'title': '上海市规划和国土资源管理局关于印发《上海市国有建设用地使用权出让预申请管理办法》及《上海市国有建设用地使用权出让预申请实施细则》的通知'}, page_content='出让人可根据相关建议和实际情况，回复意向用地人，作为进一步完善出让条件的依据。出让公告发布时，已明确的交易条件应随出让文件一并发布。八、意向用地人均可根据地块预申请公告和文件的相关规定，向出让人和交易中心提出报名申请，并承诺愿意按照相关要求参与后续出让活动。预申请可不设保证金，若确需设置预申请保证金的，可以保函形式设定。九、预申请公告期满后，交易中心将在网上发布预申请结果公告，终止预申请活动。预申请结果公告应包含以下内容：1.预申请咨询人数；2.有效预申请人数（涉及招标的除外）；3.有效预申请人受理编号（涉及招标的除外）；4.后续出让活动安排；5.其他重要提示。十、出让人在预申请结果公告中可明确有效预申请人的退出机制。退出预申请的有效预申请人一般在预申请结果公告发布后5个工作日内，完成退出手续。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-18', 'title': '金融企业国有资产转让管理办法'}, page_content='在不违反相关监督管理要求和公平竞争原则下，转让方可以对意向受让方的资质、商业信誉、行业准入、资产规模、经营情况、财务状况、管理能力等提出具体要求。第二十条 在产权交易过程中，首次挂牌价格不得低于经核准或者备案的资产评估结果。首次挂牌未能征集到意向受让方的，转让方可以根据转让标的企业情况确定新的挂牌价格并重新公告。如新的挂牌价格低于资产评估结果的90%，应当重新报批。第二十一条 经公开征集，产生2个以上（含2个）意向受让方时，转让方应当会同产权交易机构共同对意向受让方进行资格审核，根据转让标的企业的具体情况采取拍卖、招投标或者国家规定的其他公开竞价方式实施产权交易。采取拍卖方式转让非上市企业产权的，应当按照《中华人民共和国拍卖法》及其他有关规定组织实施。采取招投标方式转让非上市企业产权的，应当按照《中华人民共和国招标投标法》及其他有关规定组织实施。第二十二条 经产权交易机构公开征集只产生1个符合条件的意向受让方时，产权转让可以采取场内协议转让方式进行，但转让价格不得低于挂牌价格。采取场内协议转让方式的，转让方应当与受让方进行充分协商，依法妥善处理转让中所涉及的相关事项后，签订产权转让协议（合同，下同）。第二十三条 确定受让方后，转让方应当与受让方签订产权转让协议。转让协议应当包括下列内容：（一）转让与受让双方的名称与住所；（二）转让标的企业产权的基本情况；（三）转让方式、转让价格、价款支付时间和方式及付款条件；（四）产权交割事项；（五）转让涉及的有关税费负担；（六）协议争议的解决方式；（七）协议各方的违约责任；（八）协议变更和解除的条件；（九）转让和受让双方认为必要的其他条款。第二十四条 转让方应当按照产权转让协议的约定及时收取产权转让的全部价款，转让价款原则上应当采取货币性资产一次性收取。如金额较大、一次付清确有困难的，可以约定分期付款方式，但分期付款期限不得超过1年。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-12', 'title': '【产权交易】上海联合产权交易所有限公司企业国有产权转让登记受让意向操作细则（沪联产交〔2020〕45号)'}, page_content='第九条 信息披露公告未明确表示不接受联合受让的，多个意向受让方可组成联合受让体，向联交所提交受让申请，参与受让。联合受让体的《产权受让申请书》中应载明各成员的受让比例。第十条 信息披露公告中设定受让方资格条件的，联合受让体各成员均需满足信息披露公告中的受让方资格条件，信息披露公告明确联合体其中一方满足受让资格条件即可的除外。第十一条 联合受让体应提交《联合受让协议》，《联合受让协议》主要包括以下内容：（一）组成联合受让体的各方主体信息；（二）联合受让体拟参与受让的项目；（三）联合体内部各成员关于受让标的的分配比例及价格；（四）确定联合受让代表，授权其代表联合受让体办理受让事宜；（五）联合受让体各成员间的权利义务；（六）其他需要在联合受让协议中约定的事项。各成员应当在《联合受让协议》中约定各方对受让方所有义务承担连带责任。第十二条 联交所按照信息披露公告内容审核意向受让方受让资格，主要有以下内容：（一）意向受让方是否对其提供材料的真实性、完整性、准确性以及遵守市场规则作出承诺；（二）意向受让方的主体资格是否符合信息披露公告中的受让方资格条件的相关要求；（三）意向受让方是否已依法履行必要的内部决策程序或批准程序；（四）意向受让方是否符合信息披露公告中要求的其他条件；（五）管理层及其关联方参与受让的，是否符合法律法规相关要求；（六）受托机构是否对材料进行核实，核实意见是否明确具体。未经信息发布的受让方资格条件，不得作为确认意向受让方受让资格的依据。第十三条 登记的意向受让方不符合信息披露公告中的受让方资格条件，或提出带有附加条件的受让要求，或提交的材料不符合齐全性和规范性要求的，联交所以书面形式告知意向受让方。意向受让方应当在收到通知的次日起2个工作日内按要求做出调整。第十四条 联交所在信息披露期满后5个工作日内，就意向受让方是否符合受让条件以《受让资格反馈函》反馈至转让方。'), Document(metadata={'infosource': '信息来源：上海市', 'time': '发布时间：2021-07-15', 'title': '上海市国资委、市工商局、市产管办关于印发《上海市产权交易市场管理办法实施细则》的通知(2009年修订)'}, page_content='未经公布的受让条件不得作为确认或否定意向受让方资格的依据。在产权转让公告中公布的受让条件，一经发布不得擅自变更。因特殊原因确需变更信息公告内容的，应当由产权转让批准机构出具文件，由联交所在原信息发布渠道进行公告，并重新计算公告期。第十八条 （信息的反馈）产权交易信息发布期间，符合条件的意向受让方应当委托经纪会员向联交所提出产权受让申请。联交所应当及时受理审核，登记意向受让方的相关信息并存档，并将意向受让方的财务状况、收购资金来源以及与出让产权所在产业的关联度等信息书面告知出让方及其受托会员。联交所应当对意向受让方提交的申请及材料进行齐全性和合规性审核，经征询出让方意见后，联交所应当以书面形式将资格确认结果告知意向受让方，并抄送出让方。出让方对联交所确认的意向受让方资格有异议，应与联交所进行协商，必要时可以就有关争议事项征求政府有关职能部门意见，也可通过产权交易争端协调机制，对分歧事项进行协调。当登记的意向受让方没有响应产权转让公告中受让条件的全部要求，或提出带有附加条件的受让要求时，联交所应当及时以书面形式对其进行提示，在规定的期限内该意向受让方没有作出调整、纠正的，应取消其受让资格。信息公告超过约定期限且未征集到符合条件意向受让方的，可以按照产权转让公告的约定延长信息公告期限或者自行终结，也可以由出让方根据有关规定更改产权转让公告的内容和条件后再次公告。再次公告的期限应当不少于20个工作日。第十九条（企业国有产权的出让价格）企业国有产权出让方应当依照国家有关规定，委托具有资质的资产评估机构进行资产评估。资产评估报告经核准或者备案后，作为确定企业国有产权转让价格的主要参考依据。企业国有产权转让首次信息公告时的挂牌价不得低于经备案或者核准的转让标的资产评估结果。如在规定的公告期限内未征集到意向受让方，出让方可以在不低于评估结果90％的范围内设定新的挂牌价再次进行公告。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-18', 'title': '协议出让国有土地使用权规定'}, page_content='市、县人民政府国土资源行政主管部门公布计划接受申请的时间不得少于３０日。第九条 在公布的地段上，同一地块只有一个意向用地者的，市、县人民政府国土资源行政主管部门方可按照本规定采取协议方式出让；但商业、旅游、娱乐和商品住宅等经营性用地除外。同一地块有两个或者两个以上意向用地者的，市、县人民政府国土资源行政主管部门应当按照《招标拍卖挂牌出让国有土地使用权规定》，采取招标、拍卖或者挂牌方式出让。第十条 对符合协议出让条件的，市、县人民政府国土资源行政主管部门会同城市规划等有关部门，依据国有土地使用权出让计划、城市规划和意向用地者申请的用地项目类型、规模等，制定协议出让土地方案。协议出让土地方案应当包括拟出让地块的具体位置、界址、用途、面积、年限、土地使用条件、规划设计条件、供地时间等。第十一条 市、县人民政府国土资源行政主管部门应当根据国家产业政策和拟出让地块的情况，按照《城镇土地估价规程》的规定，对拟出让地块的土地价格进行评估，经市、县人民政府国土资源行政主管部门集体决策\ue584合理确定协议出让底价。协议出让底价不得低于协议出让最低价。协议出让底价确定后应当保密，任何单位和个人不得泄露。第十二条 协议出让土地方案和底价经有批准权的人民政府批准后，市、县人民政府国土资源行政主管部门应当与意向用地者就土地出让价格等进行充分协商，协商一致且议定的出让价格不低于出让底价的，方可达成协议。第十三条 市、县人民政府国土资源行政主管部门应当根据协议结果，与意向用地者签订《国有土地使用权出让合同》。第十四条 《国有土地使用权出让合同》签订后７日内，市、县人民政府国土资源行政主管部门应当将协议出让结果在土地有形市场等指定场所，或者通过报纸、互联网等媒介向社会公布，接受社会监督。公布协议出让结果的时间不得少于１５日。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-12', 'title': '【产权交易】上海联合产权交易所有限公司项目意向信息披露操作细则（试行）（沪联产交〔2019〕70号）'}, page_content='第一条 为规范上海联合产权交易所有限公司（以下简称联交所）项目意向信息披露行为，制定本细则。第二条意向信息披露项目转让方或增资方（以下简称意向信息披露方）应对项目材料的真实性、完整性、有效性负责。意向信息披露项目通过联交所会员等受托机构代理的，受托机构应对其代理的意向信息披露项目材料的真实性、完整性、有效性予以核实。第三条 意向信息披露项目申请材料包括：1、《意向信息披露申请书》；2、联交所认为需要提供的其他材料。第四条 项目意向信息披露期限原则上不少于5个工作日，最长不超过一年，信息披露期限自项目在联交所网站意向信息披露之次一工作日起算。第五条 意向信息披露方在意向信息披露期间需变更披露信息的，经重新申请后，在原信息发布渠道重新予以披露，信息披露期限重新计算。第六条 意向信息披露期间，意向投资方或其受托机构可以查阅意向信息披露方同意提供的转让标的（增资方）相关资料，如有其他信息查询或尽职调查需求，可与意向信息披露方协商确定。第七条 项目意向信息除在联交所官网披露外，意向信息披露方另外要求进行推介或采用定向意向信息披露方式的，可就特别要求事项在委托代理协议或相关服务合同中提出，并另行确定相应费用。第八条 意向信息披露方和意向投资方在意向信息披露期间，不得自行进行场外交易。违反本条规定的，应承担违约责任。第九条 意向信息披露期间或意向信息披露结束后，如项目转为正式披露项目，意向信息披露方应按有关交易规则办理相关手续。第十条 本细则自发布之日起施行，原《上海联合产权交易所项目意向信息披露操作规则（试行）》（沪联产交〔2013〕3号）不再执行。上海联合产权交易所有限公司2019年7月23日'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2021-07-15', 'title': '国务院国有资产监督管理委员会关于印发《企业国有产权交易操作规则》的通知'}, page_content='第二十二条 信息公告期间出现影响交易活动正常进行的情形，或者有关当事人提出中止信息公告书面申请和有关材料后，产权交易机构可以作出中止信息公告的决定。第二十三条 信息公告的中止期限由产权交易机构根据实际情况设定，一般不超过1个月。产权交易机构应当在中止期间对相关的申请事由或者争议事项进行调查核实，也可转请相关部门进行调查核实，及时作出恢复或者终结信息公告的决定。如恢复信息公告，在产权交易机构网站上的累计公告期不少于20个工作日，且继续公告的期限不少于10个工作日。第二十四条  信息公告期间出现致使交易活动无法按照规定程序正常进行的情形，并经调查核实确认无法消除时，产权交易机构可以作出终结信息公告的决定。第四章登记受让意向第二十五条 意向受让方在信息公告期限内，向产权交易机构提出产权受让申请，并提交相关材料。产权交易机构应当对意向受让方逐一进行登记。第二十六条 意向受让方可以到产权交易机构查阅产权转让标的的相关信息和材料。第二十七条 产权交易机构应当对意向受让方提交的申请及材料进行齐全性和合规性审核，并在信息公告期满后5个工作日内将意向受让方的登记情况及其资格确认意见书面告知转让方。第二十八条 转让方在收到产权交易机构的资格确认意见后，应当在5个工作日内予以书面回复。如对受让方资格条件存有异议，应当在书面意见中说明理由，并提交相关证明材料。转让方逾期未予回复的，视为同意产权交易机构作出的资格确认意见。第二十九条 经征询转让方意见后，产权交易机构应当以书面形式将资格确认结果告知意向受让方，并抄送转让方。第三十条 转让方对产权交易机构确认的意向受让方资格有异议，应当与产权交易机构进行协商，必要时可以就有关争议事项征询国有资产监督管理机构意见。第三十一条  通过资格确认的意向受让方在事先确定的时限内向产权交易机构交纳交易保证金（以到达产权交易机构指定账户为准）后获得参与竞价交易资格。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-05', 'title': '【技术交易】上海技术交易所竞价交易（拍卖）实施办法（草拟）'}, page_content='转让方及其经纪会员、拍卖机构应当在实施拍卖前，对拍卖标的进行展示或组织意向受让方勘察，并配合意向受让方查询相关情况。第十一条 意向受让方应当在技术转让公告和拍卖公告规定的期限内，与其受托经纪会员共同到主持拍卖的拍卖机构办理竞买登记手续，并通过其受托经纪会员向上海技术交易所提出技术受让申请。第十二条 意向受让方应当对提交材料的真实性、完整性、有效性负责。受托经纪会员应当对意向受让方的资信能力进行尽职调查，对意向受让方提交材料的真实性、完整性、有效性、合规性进行核实，并出具核实意见。第十三条 主持拍卖的拍卖机构应当在受理竞买登记时向意向受让方说明拍卖实施的规则和要求。主持拍卖的拍卖机构应当在竞买登记截止日后的2个工作日内将竞买登记的有关情况告知上海技术交易所，并将意向受让方登记的相关材料一并转交上海技术交易所。第十四条 上海技术交易所按照资格确认的有关程序确认意向受让方的竞买人资格，并以书面形式将资格确认结果通过经纪会员告知意向受让方、转让方和主持拍卖的拍卖机构。第十五条 约定采用拍卖方式的技术转让项目在信息发布期限内征集到两个及以上符合条件的竞买人的，由拍卖机构主持拍卖。约定采用拍卖方式的技术转让项目在信息发布期限内只征集到一个符合条件的竞买人的，不再举行拍卖，由上海技术交易所按照技术转让公告的约定组织实施交易。约定采用拍卖方式的技术转让项目在信息发布期限内未征集到符合条件的竞买人的，由上海技术交易所按照技术转让公告的约定延长信息发布期限或者终结技术交易。第十六条 竞买人应当在资格确认后的3个工作日内足额交付交易保证金，领取竞买号牌，取得合法竞买资格。交易保证金的交纳数额一般不超过技术转让挂牌价的30％。竞买人未在规定期限内交纳交易保证金的，丧失竞买资格。第十七条 竞买人可以自行参加拍卖，也可以委托代理人参加拍卖。竞买人已经委托经纪会员代理的，应当与经纪会员共同参加拍卖。'), Document(metadata={'infosource': '信息来源：国家', 'time': '发布时间：2020-11-05', 'title': '【技术交易】上海技术交易所交易暂行规则（草拟）'}, page_content='上述所有文件均须提供原件备查，如未能按上海技术交易所要求提供原件，视为无原件。第二十三条 信息披露有效期内，交易方及其会员机构应接受意向受让方的查询洽谈。意向受让方应在信息披露有效期内，直接或委托会员机构向上海技术交易所递交受让申请及相关受让文件。上海技术交易所对意向受让方的受让申请进行登记，并会同转让方按照信息披露的受让条件和国家的有关规定，对意向受让方进行资格审查。第二十四条 上海技术交易所对意向受让方所提交材料齐全性和合规性进行形式审核，并在披露公告期满后3个工作日内，将对意向受让方的交易资格审核意见书面告知转让方。第二十五条 转让方在收到上海技术交易所的交易资格审核意见后，应当在5个工作日内予以书面回复。转让方逾期未予回复的，视为同意上海技术交易所的交易资格审核意见。转让方与上海技术交易所意见不一致的，由交易行为批准单位决定意向受让方是否符合交易条件。上海技术交易所在意向受让方的交易资格确认后，通知意向受让方交纳交易保证金。第二十六条 意向受让方应当在3个工作日内将交易保证金交纳至监管机构指定的第三方结算账户，意向受让方逾期未交纳或未全额交纳交易保证金的，视为放弃交易资格。第二十七条 信息披露有效期内只产生一个符合条件的意向受让方的，可采取协议转让的方式进行交易。产生两个及以上符合条件的意向受让方的，可以采取拍卖、招投标、动态报价，或法律法规允许的其他竞价交易方式进行交易。具体交易方式由上海技术交易所与转让方协商确定。交易价格遵循市场化定价方式，鼓励各类市场主体参与交易活动，尊重交易双方的自主选择。l交易程序 组织交易和洽谈签约第二十八条 交易方提出的交易申请，在通过项目初筛及意向方主体资格形式审查后，如选择“协议转让”方式，上海技术交易所应当组织、协调，交易主体进行洽谈及调研，签署交易合同。')]

In [ ]:
'''
第一阶段召回改善方式
0、扩大召回数目；
1、提取问题关键词、实体， 每个实体随意组合以及单个实体进行多次召回；
2、对问题进行重新生成多种说法，召回多次；（备选，因为我们在做意图识别时会进行实体提取，如果提取成功率较高的话，这一步就可以省略，没必要生成多个问题）；
3、rerank排序召回，考虑第1、2点的多次召回重复较多的文档title作为一个权重排序；
'''